In [20]:
import pandas as pd

x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv')
x_label = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv')

In [21]:
print(x_train.shape)
print('-----------')
print(x_train.head(3))
print('-----------')
print(x_train.describe())
print('-----------')
print(x_train.info())
print('-----------')
print(x_train.isnull().sum())

(6499, 12)
-----------
   CustomerId   Surname  CreditScore Geography  Gender  Age  Tenure  \
0    15799217  Zetticci          791   Germany  Female   35       7   
1    15748986   Bischof          705   Germany    Male   42       8   
2    15722004    Hsiung          543    France  Female   31       4   

     Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  
0   52436.20              1          1               0        161051.75  
1  166685.92              2          1               1         55313.51  
2  138317.94              1          0               0         61843.73  
-----------
         CustomerId  CreditScore          Age       Tenure        Balance  \
count  6.499000e+03  6499.000000  6499.000000  6499.000000    6499.000000   
mean   1.569157e+07   650.396830    38.957070     5.041545   76836.581068   
std    7.187584e+04    96.618957    10.502803     2.891779   62407.570894   
min    1.556570e+07   350.000000    18.000000     0.000000       0.000000   

In [22]:
print(y_train.shape)
print(y_train.info())
print(y_train.head(5))
print(x_label.head(3))

(6499, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6499 entries, 0 to 6498
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   CustomerId  6499 non-null   int64
 1   Exited      6499 non-null   int64
dtypes: int64(2)
memory usage: 101.7 KB
None
   CustomerId  Exited
0    15799217       0
1    15748986       0
2    15722004       0
3    15780966       0
4    15636731       0
   CustomerId  Exited
0    15601012       1
1    15734762       1
2    15586757       0


In [23]:
print(x_test.shape)
print(x_test.head(3))

(3501, 12)
   CustomerId      Surname  CreditScore Geography  Gender  Age  Tenure  \
0    15601012     Abdullah          802    France  Female   60       3   
1    15734762     Ignatiev          602    France  Female   56       3   
2    15586757  Anenechukwu          801    France  Female   32       4   

     Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  
0   92887.06              1          1               0         39473.63  
1  115895.22              3          1               0          4176.17  
2   75170.54              1          1               1         37898.50  


In [24]:
print(x_train.Geography.unique())
print(x_test.Geography.unique())

['Germany' 'France' 'Spain']
['France' 'Spain' 'Germany']


In [25]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [26]:
mms_col = ['CreditScore', 'Tenure','Balance','EstimatedSalary']

for col in mms_col:
  mms = MinMaxScaler()
  x_train[col] = mms.fit_transform(x_train[mms_col])
  x_test[col] = mms.transform(x_test[mms_col])

In [27]:
le_col = ['Geography','Gender']

for col in le_col:
  le = LabelEncoder()
  x_train[col] = le.fit_transform(x_train[col])
  x_test[col] = le.transform(x_test[col])

In [29]:
ohe_dict = {}
ohe_col = ['Geography','NumOfProducts','HasCrCard','IsActiveMember']

for col in ohe_col:
  ohe = OneHotEncoder(sparse_output = False,handle_unknown ='ignore')
  ohe_x_train = ohe.fit_transform(x_train[ohe_col])
  ohe_x_test = ohe.transform(x_test[ohe_col])

In [ ]:
ohe_columns = ohe.get_feature_names_out()
ohe_x_train_df = pd.DataFrame(ohe_x_train, columns = ohe_columns)
ohe_x_train_df.index = x_train.index

x_train = pd.concat([x_train.drop(ohe_col, axis=1), ohe_x_train_df], axis=1)

In [ ]:
ohe_x_test_df = pd.DataFrame(ohe_x_test, columns = ohe_columns)
ohe_x_test_df.index = x_test.index

x_test = pd.concat([x_test.drop(ohe_col, axis=1), ohe_x_test_df], axis=1)

In [ ]:
x_df = x_train.drop(['CustomerId','Surname'], axis=1)
y_df = y_train['Exited']

In [ ]:
from sklearn.model_selection import train_test_split

train_x, val_x, train_y, val_y = train_test_split(x_df, y_df, test_size = 0.2, random_state=1234)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

params = {'n_estimators':[10,15,20,25,30],'max_depth':[5,10,15,20,50]}
model = RandomForestClassifier(random_state=1234)

gscv = GridSearchCV(estimator = model, param_grid = params, cv = 2)
gscv.fit(train_x,train_y)
print(gscv.best_params_)

{'max_depth': 10, 'n_estimators': 30}


In [ ]:
rf = RandomForestClassifier(n_estimators=30, max_depth=10,random_state=10)
rf.fit(train_x, train_y)

pred = rf.predict(val_x)

In [ ]:
from sklearn.metrics import accuracy_score , f1_score, recall_score, roc_auc_score ,precision_score

print('accuracy_score:',accuracy_score(pred, val_y))

accuracy_score: 0.8576923076923076


In [ ]:
x_test_1 = x_test.copy()
x_test_y = x_test_1.drop(['CustomerId','Surname'], axis=1)

result_1 = rf.predict(x_test_y)

print('accuracy_score:',accuracy_score(result_1, x_label['Exited']))

accuracy_score: 0.8563267637817766


In [ ]:
result = pd.DataFrame({'CustomerId':x_test['CustomerId'], 'Exited':result_1})
result.to_csv('수험번호.csv',index=False)